<span style="font-size: 5em">🦜</span>

# __LangGraph Essentials__
## Lab 7: 슈퍼바이저 멀티에이전트

## 개요

슈퍼바이저 패턴은 멀티에이전트 시스템에서 가장 기본적인 구조입니다. 하나의 **슈퍼바이저**가 들어온 요청을 분석하고, 적절한 **전문 에이전트**에게 작업을 위임합니다.

```
                    ┌─────────────────┐
                    │   슈퍼바이저    │
                    │   (분류/라우팅)  │
                    └────────┬────────┘
                             │
           ┌─────────────────┼─────────────────┐
           ▼                 ▼                 ▼
    ┌──────────┐      ┌──────────┐      ┌──────────┐
    │  정산    │      │ 웹할인   │      │  오류    │
    │ 에이전트 │      │ 에이전트 │      │ 에이전트 │
    └──────────┘      └──────────┘      └──────────┘
```

이 노트북에서는 **내부 직원 질문 분류 시스템**을 구현합니다:
- 직원이 질문을 입력하면 슈퍼바이저가 카테고리를 분류
- 분류된 카테고리에 따라 전문 에이전트가 응답 생성

<a id='imports'></a>
## 임포트

In [ ]:
from IPython.display import Image, display
from typing import Literal, TypedDict
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command
from langchain_openai import ChatOpenAI

<a id='state_definition'></a>
## State 정의

모든 노드가 공유하는 상태를 정의합니다.

In [ ]:
class AgentState(TypedDict):
    question: str      # 직원 질문
    category: str      # 분류된 카테고리
    response: str      # 최종 응답

## LLM 설정

In [ ]:
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

<a id='supervisor_node'></a>
## 슈퍼바이저 노드

슈퍼바이저는 질문을 분석하고 적절한 전문 에이전트로 라우팅합니다.
- `Command`를 사용하여 다음 노드를 동적으로 결정

In [ ]:
def supervisor(state: AgentState) -> Command[Literal["settlement", "web_discount", "error_check"]]:
    """질문을 분류하고 적절한 에이전트로 라우팅"""
    
    classification_prompt = f"""당신은 내부 직원 질문을 분류하는 분류기입니다.
다음 질문을 분석하고, 아래 3가지 카테고리 중 하나로만 답하세요:

- 정산: 정산, 수수료, 결제 대금, 매출 정산 관련 질문
- 웹할인: 웹할인 연동, 할인 적용, 프로모션 연동 관련 질문  
- 오류: 시스템 오류, 버그, 에러, 장애 관련 질문

질문: {state['question']}

카테고리(정산/웹할인/오류 중 하나만):"""
    
    result = llm.invoke(classification_prompt)
    category = result.content.strip()
    
    # 카테고리에 따라 라우팅
    route_map = {
        "정산": "settlement",
        "웹할인": "web_discount",
        "오류": "error_check"
    }
    
    next_node = route_map.get(category, "error_check")  # 기본값: error_check
    
    print(f"[슈퍼바이저] 질문 분류: {category} → {next_node} 에이전트로 전달")
    
    return Command(
        update={"category": category},
        goto=next_node
    )

<a id='specialist_agents'></a>
## 전문 에이전트 노드

각 에이전트는 해당 도메인의 전문가로서 질문에 응답합니다.

In [ ]:
def settlement_agent(state: AgentState) -> AgentState:
    """정산 관련 질문 처리"""
    
    prompt = f"""당신은 정산 업무 전문가입니다.
직원의 정산 관련 질문에 친절하고 정확하게 답변해주세요.

질문: {state['question']}

답변:"""
    
    result = llm.invoke(prompt)
    print(f"[정산 에이전트] 응답 생성 완료")
    
    return {"response": result.content}

In [ ]:
def web_discount_agent(state: AgentState) -> AgentState:
    """웹할인 연동 관련 질문 처리"""
    
    prompt = f"""당신은 웹할인 연동 업무 전문가입니다.
직원의 웹할인 연동, 프로모션 적용 관련 질문에 친절하고 정확하게 답변해주세요.

질문: {state['question']}

답변:"""
    
    result = llm.invoke(prompt)
    print(f"[웹할인 에이전트] 응답 생성 완료")
    
    return {"response": result.content}

In [ ]:
def error_agent(state: AgentState) -> AgentState:
    """오류/장애 관련 질문 처리"""
    
    prompt = f"""당신은 시스템 오류 및 장애 대응 전문가입니다.
직원의 오류, 버그, 장애 관련 질문에 친절하고 정확하게 답변해주세요.

질문: {state['question']}

답변:"""
    
    result = llm.invoke(prompt)
    print(f"[오류 에이전트] 응답 생성 완료")
    
    return {"response": result.content}

<a id='graph_building'></a>
## 그래프 구성

In [ ]:
# 그래프 빌더 생성
builder = StateGraph(AgentState)

# 노드 추가
builder.add_node("supervisor", supervisor)
builder.add_node("settlement", settlement_agent)
builder.add_node("web_discount", web_discount_agent)
builder.add_node("error_check", error_agent)

# 엣지 연결
builder.add_edge(START, "supervisor")
# supervisor는 Command로 다음 노드를 결정하므로 별도 엣지 불필요
builder.add_edge("settlement", END)
builder.add_edge("web_discount", END)
builder.add_edge("error_check", END)

# 그래프 컴파일
graph = builder.compile()

## 그래프 시각화

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

<a id='test_execution'></a>
## 테스트 실행

3가지 카테고리별로 테스트해봅니다.

In [ ]:
# 테스트 1: 정산 관련 질문
result = graph.invoke({"question": "이번 달 정산 금액이 언제 입금되나요?"})
print(f"\n📋 카테고리: {result['category']}")
print(f"💬 응답:\n{result['response']}")

In [ ]:
# 테스트 2: 웹할인 관련 질문
result = graph.invoke({"question": "신규 프로모션을 웹할인에 연동하려면 어떻게 해야 하나요?"})
print(f"\n📋 카테고리: {result['category']}")
print(f"💬 응답:\n{result['response']}")

In [ ]:
# 테스트 3: 오류 관련 질문
result = graph.invoke({"question": "주문 조회 화면에서 500 에러가 계속 발생합니다."})
print(f"\n📋 카테고리: {result['category']}")
print(f"💬 응답:\n{result['response']}")

## Takeaways

### 슈퍼바이저 패턴의 핵심

**구조:**
- 슈퍼바이저: 요청을 분석하고 적절한 에이전트로 라우팅
- 전문 에이전트: 각 도메인별 전문 처리
- `Command`: 노드 내부에서 동적으로 다음 노드 결정

**언제 사용하면 좋은가:**
- 요청을 카테고리별로 분류해야 할 때
- 각 카테고리마다 다른 처리 로직이 필요할 때
- 전문가 시스템을 구축할 때

### Try Next

- 새로운 카테고리와 전문 에이전트 추가해보기
- 슈퍼바이저가 여러 에이전트를 순차적으로 호출하도록 수정해보기
- `interrupt()`를 추가하여 사람의 승인을 받은 후 응답하도록 변경해보기